In [1]:
import os
import weaviate
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = weaviate.Client(
    url=os.environ["WEAVIATE_URL"],
    auth_client_secret=weaviate.AuthApiKey(api_key=os.environ['WEAVIATE_APIKEY']),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_APIKEY']
    }
    )

In [3]:
# Test connection
print(client.is_live())
print(client.is_ready())

True
True


#### Set you Hasura Console

#### Usecases

In [4]:
import os
from pprint import pprint

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(model="text-davinci-003",
             openai_api_key=os.environ['OPENAI_APIKEY'])

chatllm = ChatOpenAI(model="gpt-3.5-turbo",
                     openai_api_key=os.environ['OPENAI_APIKEY'])

#### Usecase 1 - Question resume

In [5]:
result = (
    client.query.get(
    "Resume", 
    [
        "application_id", 
        "content"
    ]
    )
    .with_limit(3)
    .do()
)
print(result)

{'data': {'Get': {'Resume': [{'application_id': '1800', 'content': "         ENGINEERING OPERATIONS DIRECTOR           Executive Profile    Senior Software Executive Senior Software Executive who is a key contributor to strategic planning and product development. Highly skilled at creating and implementing key software improvements and process changes by uncovering major process limitations, maximizing profitability, scalability, and competition in the global marketplace. Accomplishments (over last 15 years +) Reported directly to C-level executives and Board members, successfully bridging the gap between the business and Engineering, implementing strategic plans and ensuring that the engineering teams are aligned to business goals. Agile expert and evangelist, running software development teams for over 17 years and specifically agile software development for more that 13 years. Reorganized and transitioned many teams and companies to become smooth running agile groups, drastically re

In [10]:
resume_question_prompt_template = PromptTemplate.from_template(
    """
    You are a helpful Question Answering bot. 
    You are provided with 3 Resumes content and a question.
    Answer the question based on the content of the Resumes.
    Provide your reasoning.

    Question: {question}
    Resume1: {resume_content1} with application id {application_id1}
    Resume2: {resume_content2} with application id {application_id2}
    Resume3: {resume_content3} with application id {application_id3}

    """
)


chain = LLMChain(llm=llm, prompt=resume_question_prompt_template)

In [11]:
print(chain.run({
    "question": "Which applicant has the most experience?",
    "resume_content1": result['data']["Get"]['Resume'][0]['content'],
    "resume_content2": result['data']["Get"]['Resume'][1]['content'],
    "resume_content3": result['data']["Get"]['Resume'][2]['content'], 
    "application_id1": result['data']["Get"]['Resume'][0]['application_id'],
    "application_id2": result['data']["Get"]['Resume'][1]['application_id'],
    "application_id3": result['data']["Get"]['Resume'][2]['application_id']
}))


I don't know. While all three applicants have impressive resumes, there is not enough information given to determine which one has the most experience.


In [45]:
resume_question_prompt_template = PromptTemplate.from_template(
    """
    You are a helpful Question Answering bot. You are provided with Resume content and a question on the content.
    Answer the question from the content.
    If unsure, answer with "I don't know".

    Question: {question}
    Resume: {resume_content}

    """
)


chain = LLMChain(llm=llm, prompt=resume_question_prompt_template)


In [46]:
print(chain.run({
    "question": "How many years of experience does the candidate have?",
    "resume_content": result['data']["Get"]['Resume'][0]['content']
}))


Answer: The candidate has 17 years of experience.


In [48]:
pprint(chain.run({
    "question": "Summarise the skills of the candidate in 2-3 lines.",
    "resume_content": result['data']["Get"]['Resume'][0]['content']
}))

('\n'
 'Answer: The candidate is a dedicated teaching and counseling educator with '
 '17 years of experience. They have successfully taught English and Spanish to '
 'students of all ages, implemented an IB curriculum, administered TELPAS '
 'Online Assessments, and collaborated as a TELPAS trainer. They hold a MA in '
 'Education, Counseling Certification, and Bilingual Endorsement, and have '
 'skills such as counseling, leadership, workshops, goal setting, and working '
 'with parents.')


#### Use case - 2 But, wait I am not allowed to be querying resumes 

##### Step1: Add vector db data source on Hasura console

1. Go to postgres table and track candidate and application
2. Add Weaviate Agent by  clicking `Manage` on Data tab under Database panel on left
    - Click `Data connector Agents` 
    - Click `Add Agent`
    - Name => weaviate
    - URL => weaviate-gdc:8100
3. Connect Weaviate db 
    - Click `Connect Database`
    - Select weaviate from db selection
    - Connect Existing Database
    - Type Database name
    - Pick apiKey and host from weaviate cluster details
    


##### Step2: Define remote relationship

##### Step3: Define the permissions

1. Set relationship
    - From weaviate table -> Relationship
    - Give name `application_relationship`
    - From resume (application_id) to application (id)
2. Set permission
    - Go to Resume
    - Create new role "manager" and update permissions for "select"
        - {
            "application_relationship": {
                "hiring_manager": {
                "_eq": "x-hasura-manager-id"
                }
            }
            }
    - Select permission for columns (Toggle all)
3. Fetch by permission
    - Go to API
    - x-hasura-role : manager
    - x-hasura-manager-id: Manager2

##### Step4: Issue the select Query limit 1

#### Use case -3 Find most relevant 

In [ ]:
result = (
    client.query.get(
    "Resume", 
    [
        "application_id", 
        "content"
    ]
    )
    .with_limit(1)
    .do()
)
print(result)

1.  Resume Where -> vector -> near_text

- On Admin permission 
- Add new mutation